In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
import torch.utils.data as data
import numpy as np

# Check if CUDA is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 1: Load and Preprocess CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 to match VGG-16 input
    transforms.ToTensor(),   # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet values
])

# Load the CIFAR-10 dataset (train and test set)
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# Step 2: Load the Pre-trained VGG-16 Model (trained on ImageNet)
vgg16 = models.vgg16(pretrained=True)

# Step 3: Modify VGG-16 for Feature Extraction
# Remove the fully connected layers (fc layers) and keep only the convolutional layers
# This will output the feature maps from the last conv layer
vgg16.classifier = vgg16.classifier[:4]  # Remove the classifier part (fully connected layers)

# Move the model to GPU
vgg16.to(device)

# Set the model to evaluation mode
vgg16.eval()

# Function to extract features
def extract_features(dataloader, model, device):
    features = []
    labels = []

    with torch.no_grad():  # Disable gradient computation for inference
        for inputs, targets in dataloader:
            inputs = inputs.to(device)

            # Forward pass through the VGG-16 model
            outputs = model(inputs)

            # Flatten the output from conv layers to (batch_size, feature_dim)
            outputs = outputs.view(outputs.size(0), -1)

            features.append(outputs.cpu())  # Move to CPU and store
            labels.append(targets)

    # Concatenate all features and labels
    features = torch.cat(features)
    labels = torch.cat(labels)

    return features, labels

# Step 4: Extract features for the training set and test set
train_features, train_labels = extract_features(train_loader, vgg16, device)
test_features, test_labels = extract_features(test_loader, vgg16, device)

# Check the shape of the extracted features
print(f"Training features shape: {train_features.shape}")
print(f"Test features shape: {test_features.shape}")

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


c:\Users\Test\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Test\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training features shape: torch.Size([50000, 4096])
Test features shape: torch.Size([10000, 4096])


In [11]:
train_dataset[0]

(tensor([[[-1.1075, -1.1075, -1.1075,  ...,  0.4166,  0.4166,  0.4166],
          [-1.1075, -1.1075, -1.1075,  ...,  0.4166,  0.4166,  0.4166],
          [-1.1075, -1.1075, -1.1075,  ...,  0.4166,  0.4166,  0.4166],
          ...,
          [ 0.9132,  0.9132,  0.9132,  ..., -0.0116, -0.0116, -0.0116],
          [ 0.9132,  0.9132,  0.9132,  ..., -0.0116, -0.0116, -0.0116],
          [ 0.9132,  0.9132,  0.9132,  ..., -0.0116, -0.0116, -0.0116]],
 
         [[-0.9503, -0.9503, -0.9503,  ...,  0.1352,  0.1352,  0.1352],
          [-0.9503, -0.9503, -0.9503,  ...,  0.1352,  0.1352,  0.1352],
          [-0.9503, -0.9503, -0.9503,  ...,  0.1352,  0.1352,  0.1352],
          ...,
          [ 0.4853,  0.4853,  0.4853,  ..., -0.4251, -0.4251, -0.4251],
          [ 0.4853,  0.4853,  0.4853,  ..., -0.4251, -0.4251, -0.4251],
          [ 0.4853,  0.4853,  0.4853,  ..., -0.4251, -0.4251, -0.4251]],
 
         [[-0.7064, -0.7064, -0.7064,  ..., -0.0092, -0.0092, -0.0092],
          [-0.7064, -0.7064,

In [8]:
train_features = train_features.cpu().detach().numpy()
test_features = test_features.cpu().detach().numpy()
train_labels = train_labels.cpu().detach().numpy()
test_labels = test_labels.cpu().detach().numpy()

## VALIDATION SET

In [ ]:
val_features = train_features[-5000:,:]
train_features = train_features[:-5000,:]
val_labels = train_labels[-5000:]
train_labels = train_labels[:-5000]

In [9]:
torch.cuda.empty_cache()

In [8]:
#np.save(r'C:\Users\Test\Desktop\p7\Spectral\features\train_features_vgg16_cifar10.npy', train_features)
#np.save(r'C:\Users\Test\Desktop\p7\Spectral\features\test_features_vgg16_cifar10.npy', test_features)
#np.save(r'C:\Users\Test\Desktop\p7\Spectral\features\train_labels_vgg16_cifar10.npy', train_labels)
#np.save(r'C:\Users\Test\Desktop\p7\Spectral\features\test_labels_vgg16_cifar10.npy', test_labels)